# TFM - Modelado de Tópicos con BERTopic

Este notebook implementa el modelado de tópicos para reviews de restaurantes usando técnicas modernas de aprendizaje automático.

**Título del TFM**: Análisis de Datos y Procesamiento de Lenguaje Natural para la Extracción de Opiniones y Modelado de Tópicos en Restaurantes: Un Enfoque de Big Data y Ciencia de Datos Aplicado al Estudio Integral del Sector Gastronómico

## Objetivo del Notebook
Implementar un pipeline de modelado de tópicos que:
1. Cargue las reviews de restaurantes con análisis de sentimiento
2. Aplique BERTopic para descubrir temas latentes automáticamente
3. Genere visualizaciones interactivas de los tópicos encontrados
4. Analice la relación entre tópicos y sentimientos
5. Identifique tendencias y patrones temáticos en el sector gastronómico

## Modelo Utilizado
- **Modelo**: BERTopic con embeddings de sentence-transformers
- **Embeddings**: `all-MiniLM-L6-v2` para representación semántica
- **Clustering**: HDBSCAN para agrupación de documentos similares
- **Reducción dimensionalidad**: UMAP para visualización

## Características del Análisis
- Detección automática del número óptimo de tópicos
- Visualizaciones interactivas con plotly
- Análisis temporal de evolución de tópicos
- Correlación entre tópicos y sentimientos
- Extracción de palabras clave representativas
- Identificación de tópicos emergentes y declinantes

## 1. Instalación y Configuración

### Librerías Requeridas

Para el modelado de tópicos necesitamos las siguientes librerías principales:

- **bertopic**: Framework moderno para modelado de tópicos
- **sentence-transformers**: Embeddings semánticos de alta calidad
- **umap-learn**: Reducción de dimensionalidad para visualización
- **hdbscan**: Clustering jerárquico basado en densidad
- **scikit-learn**: Utilidades adicionales de machine learning
- **pandas**: Manipulación y análisis de datos
- **matplotlib/seaborn**: Visualizaciones estáticas
- **plotly**: Visualizaciones interactivas
- **wordcloud**: Nubes de palabras para representación visual

### Ventajas de BERTopic sobre LDA Tradicional

- **Embeddings contextuales**: Mejor comprensión semántica
- **Clustering dinámico**: Número de tópicos determinado automáticamente
- **Visualizaciones modernas**: Gráficos interactivos y dinámicos
- **Escalabilidad**: Eficiente para datasets grandes
- **Interpretabilidad**: Representaciones más coherentes de tópicos

In [1]:
# Instalación de dependencias para modelado de tópicos
# Descomenta las siguientes líneas si necesitas instalar las librerías
# !uv add bertopic sentence-transformers umap-learn hdbscan scikit-learn
# !uv add matplotlib seaborn plotly wordcloud nltk spacy

In [2]:
# Imports necesarios para el modelado de tópicos
import pandas as pd
import numpy as np
import torch
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from wordcloud import WordCloud
import re
import nltk
from tqdm.auto import tqdm
import warnings
import os
warnings.filterwarnings('ignore')

# Crear directorio para guardar figuras
os.makedirs('../../figures/complete_analysis', exist_ok=True)

# Configuración de visualización
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Detectar dispositivo de procesamiento (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo de procesamiento: {device}")
print(f"Versión de PyTorch: {torch.__version__}")

# Configurar sentence-transformers para usar GPU si está disponible
if torch.cuda.is_available():
    print(f"GPU detectada: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("Sentence-transformers usará GPU para embeddings")
else:
    print("Usando CPU para procesamiento de embeddings")

print("Configuración completada exitosamente")
print(f"Directorio de figuras creado: ../../figures/complete_analysis/")

Dispositivo de procesamiento: cuda
Versión de PyTorch: 2.7.1+cu126
GPU detectada: NVIDIA GeForce RTX 4080
Memoria GPU disponible: 16.9 GB
Sentence-transformers usará GPU para embeddings
Configuración completada exitosamente
Directorio de figuras creado: ../../figures/complete_analysis/


## 2. Carga de Datos con Análisis de Sentimiento

### Archivo de Entrada

Cargaremos el archivo CSV generado por el análisis de sentimiento previo que contiene:
- **Texto original** de las reseñas de restaurantes (`text`)
- **Sentimientos clasificados** (`predicted_sentiment`: POSITIVE, NEGATIVE, NEUTRAL)
- **Confianza del modelo** (`confidence`) para filtrar predicciones de alta calidad
- **Sentimientos esperados** (`expected_sentiment`) basados en calificaciones
- **Calificaciones originales** (`stars`) para análisis comparativo
- **Metadatos** adicionales como fechas, IDs de usuario y negocio

### Estructura del Dataset CSV

El archivo `sentiment_analysis.csv` contiene las siguientes columnas:
- `review_id`: Identificador único de la reseña
- `user_id`: Identificador único del usuario
- `business_id`: Identificador único del restaurante
- `stars`: Calificación en estrellas (1-5)
- `useful`, `funny`, `cool`: Métricas de engagement
- `text`: Contenido textual de la reseña
- `date`: Fecha de la reseña
- `expected_sentiment`: Sentimiento esperado basado en estrellas
- `predicted_sentiment`: Sentimiento predicho por RoBERTa
- `confidence`: Confianza del modelo (0-1)
- `score_negative`, `score_neutral`, `score_positive`: Scores detallados

### Preparación para Modelado de Tópicos

El modelado de tópicos se beneficia de:
- **Textos con alta confianza** del modelo de sentimiento para asegurar calidad
- **Longitud mínima** de reviews para garantizar contenido semántico suficiente
- **Filtrado de outliers** para mejorar la coherencia de tópicos
- **Balanceo por sentimiento** para análisis comparativo entre categorías

### Estrategia de Muestreo

Para eficiencia computacional y calidad de resultados:
- Selección estratificada por sentimiento predicho
- Filtrado por confianza del modelo (≥ 0.7 recomendado)
- Filtrado por longitud mínima de texto (≥ 50 caracteres)
- Eliminación de reviews duplicadas o muy similares
- Priorización de reviews con alta utilidad/engagement

In [3]:
# Cargar datos de reviews con análisis de sentimiento previo
print("Iniciando carga de datos de reviews con análisis de sentimiento...")

# Intentar cargar desde diferentes rutas del proyecto
try:
    df_reviews = pd.read_csv('../../data/results/sentiment_analysis.csv')
    print("Datos cargados exitosamente desde '../../data/results/sentiment_analysis.csv'")
    ruta_carga = "../../data/results/sentiment_analysis.csv"
except:
    try:
        df_reviews = pd.read_csv('../data/results/sentiment_analysis.csv')
        print("Datos cargados exitosamente desde '../data/results/sentiment_analysis.csv'")
        ruta_carga = "../data/results/sentiment_analysis.csv"
    except:
        try:
            df_reviews = pd.read_csv('data/results/sentiment_analysis.csv')
            print("Datos cargados exitosamente desde 'data/results/sentiment_analysis.csv'")
            ruta_carga = "data/results/sentiment_analysis.csv"
        except:
            print("ERROR: No se pudo cargar el archivo de análisis de sentimiento")
            print("SOLUCIÓN: Ejecuta primero el notebook de análisis de sentimiento")
            print("O asegúrate de que el archivo esté en alguna de estas rutas:")
            print("  - ../../data/results/sentiment_analysis.csv")
            print("  - ../data/results/sentiment_analysis.csv")
            print("  - data/results/sentiment_analysis.csv")
            raise FileNotFoundError("Archivo de análisis de sentimiento no encontrado")

print(f"Dataset cargado correctamente: {len(df_reviews):,} reviews")
print(f"Ruta utilizada: {ruta_carga}")
print(f"Columnas disponibles: {list(df_reviews.columns)}")

# Verificar que las columnas de sentimiento estén presentes
required_columns = ['predicted_sentiment', 'confidence', 'text']
missing_columns = [col for col in required_columns if col not in df_reviews.columns]

if missing_columns:
    print(f"WARNING: Faltan columnas requeridas: {missing_columns}")
    print("Ejecuta primero el notebook de análisis de sentimiento")
else:
    print("Columnas de sentimiento verificadas correctamente")
    
# Verificar estructura del dataset
print(f"\nEstructura del dataset:")
print(f"- Total de reviews: {len(df_reviews):,}")
print(f"- Memoria utilizada: {df_reviews.memory_usage(deep=True).sum() / 1e6:.1f} MB")

# Mostrar distribución de sentimientos
if 'predicted_sentiment' in df_reviews.columns:
    print(f"\nDistribución de sentimientos predichos:")
    sentiment_dist = df_reviews['predicted_sentiment'].value_counts()
    for sentiment, count in sentiment_dist.items():
        percentage = (count / len(df_reviews)) * 100
        print(f"- {sentiment}: {count:,} ({percentage:.1f}%)")

# Verificar calidad de datos
print(f"\nCalidad de datos:")
print(f"- Reviews sin texto: {df_reviews['text'].isnull().sum():,}")
print(f"- Reviews con texto vacío: {(df_reviews['text'].str.len() == 0).sum():,}")
print(f"- Confianza promedio del modelo: {df_reviews['confidence'].mean():.3f}")

print("\nDataset listo para modelado de tópicos!")

Iniciando carga de datos de reviews con análisis de sentimiento...
Datos cargados exitosamente desde '../../data/results/sentiment_analysis.csv'
Dataset cargado correctamente: 100,000 reviews
Ruta utilizada: ../../data/results/sentiment_analysis.csv
Columnas disponibles: ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date', 'expected_sentiment', 'predicted_sentiment', 'confidence', 'score_negative', 'score_neutral', 'score_positive']
Columnas de sentimiento verificadas correctamente

Estructura del dataset:
- Total de reviews: 100,000
- Memoria utilizada: 111.0 MB

Distribución de sentimientos predichos:
- POSITIVE: 74,472 (74.5%)
- NEGATIVE: 20,408 (20.4%)
- NEUTRAL: 5,120 (5.1%)

Calidad de datos:
- Reviews sin texto: 0
- Reviews con texto vacío: 0
- Confianza promedio del modelo: 0.875

Dataset listo para modelado de tópicos!


## 3. Exploración Inicial del Dataset con Sentimientos

### Análisis de Calidad de Datos

Analizaremos la estructura y características del dataset cargado:
- **Distribución de sentimientos**: Verificar balance entre POSITIVE, NEGATIVE, NEUTRAL
- **Confianza del modelo**: Análisis de la calidad de las predicciones
- **Longitud de textos**: Estadísticas de longitud para filtrado óptimo
- **Correlación con estrellas**: Validación de coherencia entre sentimientos y calificaciones

### Criterios de Filtrado para Topic Modeling

Para obtener tópicos de alta calidad aplicaremos:
- **Filtrado por confianza**: ≥ 0.7 para asegurar predicciones confiables
- **Filtrado por longitud**: ≥ 50 caracteres para contenido semántico suficiente
- **Eliminación de duplicados**: Evitar sesgo por reviews repetidas
- **Muestreo estratificado**: Balance entre sentimientos para análisis comparativo

### Optimización Computacional

Considerando el tamaño del dataset (>100K reviews):
- **Muestra representativa**: Máximo 200,000 reviews para eficiencia
- **Estratificación**: Mantener proporción por sentimientos
- **Calidad vs cantidad**: Priorizar reviews de alta confianza

In [4]:
# Preparación de datos para modelado de tópicos
print("Iniciando preparación de datos para modelado de tópicos...")

# Filtrar reviews por confianza del modelo de sentimiento (≥ 0.7)
confidence_threshold = 0.7
df_high_confidence = df_reviews[df_reviews['confidence'] >= confidence_threshold].copy()
print(f"Después de filtrar por confianza (≥ {confidence_threshold}): {len(df_high_confidence):,} reviews")

# Filtrar reviews por longitud mínima (al menos 50 caracteres)
df_high_confidence['text_length'] = df_high_confidence['text'].str.len()
min_length = 50
df_filtered = df_high_confidence[df_high_confidence['text_length'] >= min_length].copy()
print(f"Después de filtrar por longitud mínima ({min_length} chars): {len(df_filtered):,} reviews")

# Eliminar reviews duplicadas por texto
df_filtered = df_filtered.drop_duplicates(subset=['text'], keep='first')
print(f"Después de eliminar duplicados: {len(df_filtered):,} reviews")

# Crear muestra estratificada por sentimiento para análisis eficiente
sample_size = min(200000, len(df_filtered))  # Máximo 200,000 reviews
print(f"Tamaño de muestra objetivo: {sample_size:,} reviews")

if 'predicted_sentiment' in df_filtered.columns:
    # Muestreo estratificado por sentimiento predicho
    sentiment_counts = df_filtered['predicted_sentiment'].value_counts()
    samples_per_sentiment = sample_size // len(sentiment_counts)
    
    df_sample = df_filtered.groupby('predicted_sentiment', group_keys=False).apply(
        lambda x: x.sample(min(len(x), samples_per_sentiment), random_state=42)
    ).reset_index(drop=True)
else:
    # Muestreo aleatorio simple si no hay columna de sentimiento
    df_sample = df_filtered.sample(n=sample_size, random_state=42).reset_index(drop=True)

print(f"Muestra final para modelado de tópicos: {len(df_sample):,} reviews")

# Verificar distribución de sentimientos en la muestra
if 'predicted_sentiment' in df_sample.columns:
    print(f"\nDistribución de sentimientos en la muestra:")
    sample_sentiment_dist = df_sample['predicted_sentiment'].value_counts()
    print(sample_sentiment_dist)
    
    # Mostrar porcentajes
    for sentiment, count in sample_sentiment_dist.items():
        percentage = (count / len(df_sample)) * 100
        print(f"- {sentiment}: {count:,} reviews ({percentage:.1f}%)")

# Verificar distribución de confianza en la muestra
print(f"\nEstadísticas de confianza en la muestra:")
print(f"- Confianza promedio: {df_sample['confidence'].mean():.3f}")
print(f"- Confianza mediana: {df_sample['confidence'].median():.3f}")
print(f"- Confianza mínima: {df_sample['confidence'].min():.3f}")
print(f"- Confianza máxima: {df_sample['confidence'].max():.3f}")

print("\nMuestra preparada exitosamente para modelado de tópicos!")

Iniciando preparación de datos para modelado de tópicos...
Después de filtrar por confianza (≥ 0.7): 84,788 reviews
Después de filtrar por longitud mínima (50 chars): 84,687 reviews
Después de eliminar duplicados: 84,680 reviews
Tamaño de muestra objetivo: 84,680 reviews
Muestra final para modelado de tópicos: 43,993 reviews

Distribución de sentimientos en la muestra:
predicted_sentiment
POSITIVE    28226
NEGATIVE    15067
NEUTRAL       700
Name: count, dtype: int64
- POSITIVE: 28,226 reviews (64.2%)
- NEGATIVE: 15,067 reviews (34.2%)
- NEUTRAL: 700 reviews (1.6%)

Estadísticas de confianza en la muestra:
- Confianza promedio: 0.919
- Confianza mediana: 0.947
- Confianza mínima: 0.700
- Confianza máxima: 0.992

Muestra preparada exitosamente para modelado de tópicos!


In [5]:
# Preprocesamiento mínimo del texto para BERTopic
def preprocesar_texto_minimal(text):
    """
    Preprocesamiento mínimo para BERTopic
    Solo limpieza básica sin alterar la semántica
    
    Args:
        text (str): Texto original
    
    Returns:
        str: Texto con limpieza mínima
    """
    if pd.isna(text):
        return ""
    
    # Convertir a string si no lo es
    text = str(text)
    
    # Eliminar caracteres de control
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', ' ', text)
    
    # Normalizar espacios múltiples
    text = re.sub(r'\s+', ' ', text)
    
    # Eliminar espacios al inicio y final
    text = text.strip()
    
    return text

# Aplicar preprocesamiento mínimo
print("Aplicando preprocesamiento mínimo...")
df_sample['text_clean'] = df_sample['text'].apply(preprocesar_texto_minimal)

# Filtrar textos que quedaron vacíos después del preprocesamiento
df_sample = df_sample[df_sample['text_clean'].str.len() > 0].reset_index(drop=True)
print(f"Después del preprocesamiento: {len(df_sample):,} reviews")

# Verificar el resultado del preprocesamiento
print(f"\nEjemplo de preprocesamiento:")
sample_idx = 0
print(f"Texto original: {df_sample.iloc[sample_idx]['text'][:200]}...")
print(f"Texto procesado: {df_sample.iloc[sample_idx]['text_clean'][:200]}...")

# Preparar lista de documentos para BERTopic
documents = df_sample['text_clean'].tolist()
print(f"\nDocumentos preparados para modelado: {len(documents):,}")

Aplicando preprocesamiento mínimo...
Después del preprocesamiento: 43,993 reviews

Ejemplo de preprocesamiento:
Texto original: Um...well what can I say?  They are traditional Japanese style restaurant, however, they have very average sushi (I don't even think the fish is that fresh, at least it didn't smell/taste that way).  ...
Texto procesado: Um...well what can I say? They are traditional Japanese style restaurant, however, they have very average sushi (I don't even think the fish is that fresh, at least it didn't smell/taste that way). I ...

Documentos preparados para modelado: 43,993


## 4. Configuración del Modelo BERTopic

### Componentes del Pipeline

BERTopic utiliza un pipeline modular que incluye:

1. **Embedding Model**: Sentence-transformers para representación semántica
2. **Dimensionality Reduction**: UMAP para reducir dimensionalidad
3. **Clustering**: HDBSCAN para agrupar documentos similares
4. **Vectorizer**: CountVectorizer para representación de tópicos

### Configuración Personalizada

Para nuestro dataset de reviews de restaurantes:
- **Embedding model**: `all-MiniLM-L6-v2` (balance entre calidad y velocidad)
- **UMAP**: Configurado para preservar estructura local y global
- **HDBSCAN**: Parámetros ajustados para detectar tópicos coherentes
- **Vocabulario**: Filtrado para términos relevantes del dominio gastronómico

### Parámetros Optimizados

- **min_cluster_size**: 50 documentos mínimo para evitar tópicos de ruido
- **n_neighbors**: 15 para balance entre estructura local y global
- **n_components**: 5 dimensiones para clustering efectivo
- **n_gram_range**: (1,2) para capturar unigramas y bigramas significativos
- **max_features**: 5000 características para vocabulario optimizado

In [6]:
# Configuración del modelo de embeddings
print("Configurando modelo de embeddings...")

# Usar sentence-transformer para análisis semántico
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
print(f"Modelo de embeddings cargado: all-MiniLM-L6-v2")
print(f"Dispositivo: {device}")

# Configuración de UMAP para reducción de dimensionalidad
umap_model = UMAP(
    n_neighbors=15,          # Balance entre estructura local y global
    n_components=5,          # Dimensiones reducidas para clustering
    min_dist=0.0,           # Permite agrupaciones más compactas
    metric='cosine',        # Métrica apropiada para embeddings de texto
    random_state=42         # Reproducibilidad
)

# Configuración de HDBSCAN para clustering
hdbscan_model = HDBSCAN(
    min_cluster_size=50,    # Mínimo 50 documentos por tópico
    metric='euclidean',     # Métrica para el espacio reducido
    cluster_selection_method='eom',  # Excess of Mass para mejor separación
    prediction_data=True    # Permitir predicciones en nuevos documentos
)

# Configuración del vectorizador para representación de tópicos
vectorizer_model = CountVectorizer(
    ngram_range=(1, 2),    # Unigramas y bigramas
    stop_words='english',   # Eliminar stop words en inglés
    min_df=5,              # Mínimo 5 apariciones
    max_df=0.95,           # Máximo 95% de documentos
    max_features=5000      # Máximo 5000 características
)

print("Componentes configurados:")
print(f"- UMAP: n_neighbors={umap_model.n_neighbors}, n_components={umap_model.n_components}")
print(f"- HDBSCAN: min_cluster_size={hdbscan_model.min_cluster_size}")
print(f"- Vectorizer: ngram_range={vectorizer_model.ngram_range}, max_features={vectorizer_model.max_features}")

Configurando modelo de embeddings...
Modelo de embeddings cargado: all-MiniLM-L6-v2
Dispositivo: cuda
Componentes configurados:
- UMAP: n_neighbors=15, n_components=5
- HDBSCAN: min_cluster_size=50
- Vectorizer: ngram_range=(1, 2), max_features=5000


In [7]:
# Crear modelo BERTopic con configuración personalizada
print("Iniciando creación del modelo BERTopic...")

topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    language='english',     # Idioma para stop words
    calculate_probabilities=True,  # Calcular probabilidades de asignación
    verbose=True           # Mostrar progreso
)

print("Modelo BERTopic creado exitosamente")
print(f"Configuración:")
print(f"- Embedding model: all-MiniLM-L6-v2")
print(f"- Top words por tópico: 10")
print(f"- Cálculo de probabilidades: Activado")
print(f"- Documentos a procesar: {len(documents):,}")

# Estimación de tiempo de procesamiento
embedding_time_per_doc = 0.001  # ~1ms por documento con GPU
clustering_time = 30  # ~30 segundos para clustering
total_time_estimate = (len(documents) * embedding_time_per_doc) + clustering_time

print(f"\nEstimación de tiempo de procesamiento:")
print(f"- Generación de embeddings: ~{len(documents) * embedding_time_per_doc:.0f} segundos")
print(f"- Clustering y reducción dimensional: ~{clustering_time} segundos")
print(f"- Tiempo total estimado: ~{total_time_estimate/60:.1f} minutos")

Iniciando creación del modelo BERTopic...
Modelo BERTopic creado exitosamente
Configuración:
- Embedding model: all-MiniLM-L6-v2
- Top words por tópico: 10
- Cálculo de probabilidades: Activado
- Documentos a procesar: 43,993

Estimación de tiempo de procesamiento:
- Generación de embeddings: ~44 segundos
- Clustering y reducción dimensional: ~30 segundos
- Tiempo total estimado: ~1.2 minutos


## 5. Ejecución del Modelado de Tópicos

### Proceso de Entrenamiento

En esta sección ejecutaremos el modelo BERTopic para descubrir tópicos automáticamente:
1. **Generación de embeddings**: Conversión de texto a representaciones vectoriales
2. **Reducción dimensional**: UMAP para proyectar a espacio de menor dimensión
3. **Clustering**: HDBSCAN para agrupar documentos similares
4. **Extracción de tópicos**: Identificación de palabras clave representativas

### Métricas de Evaluación

Analizaremos:
- **Número de tópicos encontrados**: Validar detección automática
- **Coherencia de tópicos**: Calidad semántica de agrupaciones
- **Distribución de documentos**: Balance entre tópicos
- **Palabras clave**: Representatividad y relevancia

### Tiempo de Procesamiento

El proceso tomará aproximadamente 1-2 minutos con GPU para ~44,000 documentos.

In [8]:
# Ejecutar el modelado de tópicos
print("Iniciando entrenamiento del modelo BERTopic...")
print(f"Procesando {len(documents):,} documentos...")

# Medir tiempo de ejecución
import time
start_time = time.time()

# Ejecutar BERTopic (fit_transform)
topics, probabilities = topic_model.fit_transform(documents)

# Calcular tiempo transcurrido
elapsed_time = time.time() - start_time
print(f"Tiempo de entrenamiento: {elapsed_time/60:.2f} minutos")

# Información sobre los tópicos encontrados
topic_info = topic_model.get_topic_info()
num_topics = len(topic_info) - 1  # -1 porque el tópico -1 es para outliers

print(f"\nModelado de tópicos completado exitosamente!")
print(f"Resultados:")
print(f"- Tópicos encontrados: {num_topics}")
print(f"- Documentos procesados: {len(topics):,}")
print(f"- Documentos sin asignar (outliers): {sum(t == -1 for t in topics):,}")
print(f"- Documentos asignados a tópicos: {sum(t != -1 for t in topics):,}")

# Mostrar distribución de documentos por tópico (top 10)
print(f"\nTop 10 tópicos por número de documentos:")
topic_counts = topic_info.head(11)  # 11 porque incluye el tópico -1
for idx, row in topic_counts.iterrows():
    if row['Topic'] != -1:  # Excluir outliers
        print(f"Tópico {row['Topic']:2d}: {row['Count']:5,} docs - Palabras clave: {', '.join(row['Representation'][:5])}")

print(f"\nModelado de tópicos listo para análisis y visualización!")

2025-07-11 01:15:43,818 - BERTopic - Embedding - Transforming documents to embeddings.


Iniciando entrenamiento del modelo BERTopic...
Procesando 43,993 documentos...


Batches:   0%|          | 0/1375 [00:00<?, ?it/s]

2025-07-11 01:15:57,938 - BERTopic - Embedding - Completed ✓
2025-07-11 01:15:57,938 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-11 01:16:30,968 - BERTopic - Dimensionality - Completed ✓
2025-07-11 01:16:30,970 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-11 01:16:38,075 - BERTopic - Cluster - Completed ✓
2025-07-11 01:16:38,080 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-11 01:16:41,679 - BERTopic - Representation - Completed ✓


Tiempo de entrenamiento: 0.98 minutos

Modelado de tópicos completado exitosamente!
Resultados:
- Tópicos encontrados: 70
- Documentos procesados: 43,993
- Documentos sin asignar (outliers): 14,575
- Documentos asignados a tópicos: 29,418

Top 10 tópicos por número de documentos:
Tópico  0: 5,888 docs - Palabras clave: minutes, asked, manager, told, rude
Tópico  1: 2,395 docs - Palabras clave: tacos, mexican, taco, salsa, burrito
Tópico  2: 2,024 docs - Palabras clave: food great, great food, great service, server, good food
Tópico  3: 1,884 docs - Palabras clave: pizza, crust, pizzas, best pizza, pepperoni
Tópico  4: 1,201 docs - Palabras clave: sushi, roll, rolls, best sushi, hibachi
Tópico  5: 1,092 docs - Palabras clave: burger, fries, burgers, bun, ketchup
Tópico  6:   863 docs - Palabras clave: italian, pasta, italian food, sauce, ravioli
Tópico  7:   843 docs - Palabras clave: chinese, chinese food, rice, noodles, dumplings
Tópico  8:   752 docs - Palabras clave: beer, beers, se

## 6. Análisis Detallado de Tópicos

### Exploración de Tópicos Descubiertos

Analizaremos en detalle los tópicos encontrados para entender:
- **Temáticas principales**: Qué aspectos de restaurantes cubren los tópicos
- **Coherencia semántica**: Calidad de la agrupación temática
- **Representatividad**: Qué tan bien representan las palabras clave cada tópico
- **Distribución**: Balance entre tópicos en términos de documentos

### Análisis de Palabras Clave

Para cada tópico examinaremos:
- **Top palabras**: Las palabras más representativas
- **Relevancia**: Importancia relativa dentro del tópico
- **Contexto gastronómico**: Relación con aspectos de restaurantes

### Ejemplos Representativos

Seleccionaremos reviews ejemplo que mejor representen cada tópico principal.

In [9]:
# Análisis detallado de los tópicos encontrados
print("ANÁLISIS DETALLADO DE TÓPICOS")
print("=" * 50)

# Mostrar información completa de todos los tópicos
print(f"Información completa de tópicos:")
display(topic_info)

# Análisis de los tópicos principales (top 10)
print(f"\nANÁLISIS DE LOS 10 TÓPICOS PRINCIPALES")
print("-" * 45)

top_topics = topic_info[topic_info['Topic'] != -1].head(10)

for idx, row in top_topics.iterrows():
    topic_id = row['Topic']
    count = row['Count']
    words = row['Representation']
    
    print(f"\nTÓPICO {topic_id} ({count:,} documentos)")
    print(f"   Palabras clave: {', '.join(words[:10])}")
    
    # Obtener documentos más representativos de este tópico
    topic_docs = topic_model.get_representative_docs(topic_id)
    if topic_docs:
        print(f"   Ejemplo representativo:")
        example_text = topic_docs[0][:200] + "..." if len(topic_docs[0]) > 200 else topic_docs[0]
        print(f"   '{example_text}'")

# Estadísticas generales
print(f"\nESTADÍSTICAS GENERALES")
print("-" * 25)
docs_asignados = sum(t != -1 for t in topics)
docs_outliers = sum(t == -1 for t in topics)
print(f"Total de tópicos válidos: {num_topics}")
print(f"Documentos promedio por tópico: {docs_asignados / num_topics:.1f}")
print(f"Porcentaje de outliers: {docs_outliers / len(topics) * 100:.1f}%")

# Analizar distribución de tamaños de tópicos
topic_sizes = topic_info[topic_info['Topic'] != -1]['Count']
print(f"Tamaño del tópico más grande: {topic_sizes.max():,}")
print(f"Tamaño del tópico más pequeño: {topic_sizes.min():,}")
print(f"Mediana de tamaño de tópicos: {topic_sizes.median():.0f}")

ANÁLISIS DETALLADO DE TÓPICOS
Información completa de tópicos:


,Topic,Count,Name,Representation,Representative_Docs
0,-1,14575,-1_sandwich_shrimp_bread_sauce,"[sandwich, shrimp, bread, sauce, fries, fried, pork, fish, beer, server]",[Our lunch at Vernick Fish was everything we'd hoped it would be - with a few surprises! Food wa...
1,0,5888,0_minutes_asked_manager_told,"[minutes, asked, manager, told, rude, waitress, customer, server, waited, waiting]",[Went to Joseph Ambler Inn over the weekend as it is usually a very good experience with excelle...
2,1,2395,1_tacos_mexican_taco_salsa,"[tacos, mexican, taco, salsa, burrito, mexican food, chips, margaritas, guacamole, tortillas]",[We used to enjoy Tortilla Press. We recently went there for lunch. We found it very underwhelmi...
3,2,2024,2_food great_great food_great service_server,"[food great, great food, great service, server, good food, attentive, love place, wonderful, foo...",[Great Food. Great Service. Great prices and for many many years now. GREAT WORK TEAM. BOOM. =v ...
4,3,1884,3_pizza_crust_pizzas_best pizza,"[pizza, crust, pizzas, best pizza, pepperoni, toppings, pie, slice, sauce, pizza place]",[Giovanni's pizza is the best pizza shop in town. We always stop here to get a delicious pizza w...
...,...,...,...,...,...
66,65,63,65_gluten_gluten free_free_gf,"[gluten, gluten free, free, gf, bread, dairy, vegetarian, crepe, rice, option]","[Overall, this was a disappointment. The menu online indicated some items were gluten free which..."
67,66,62,66_tea_boba_bubble_teas,"[tea, boba, bubble, teas, milk, flavors, green tea, sweet, sweetness, smoothies]","[""Best boba tea I've ever had,"" says my friend who spent 4 years in LA This I had to taste for m..."
68,67,57,67_french_french quarter_quarter_dined,"[french, french quarter, quarter, dined, starter, cafe, beach, jazz, outstanding, entree]",[This cafe is a quick uber drive away from the French quarter and well worth it. It's located in...
69,68,52,68_st pete_pete_st_beach,"[st pete, pete, st, beach, downtown, restaurants, wow, fish chips, fish, selection]","[As a St. Pete native, I can confidently say this is the best breakfast place in all of St. Pete..."



ANÁLISIS DE LOS 10 TÓPICOS PRINCIPALES
---------------------------------------------

TÓPICO 0 (5,888 documentos)
   Palabras clave: minutes, asked, manager, told, rude, waitress, customer, server, waited, waiting
   Ejemplo representativo:
   'Went to Joseph Ambler Inn over the weekend as it is usually a very good experience with excellent food. Arrived and waited at front reception area for about 5 minutes. Was seated in an area with about...'

TÓPICO 1 (2,395 documentos)
   Palabras clave: tacos, mexican, taco, salsa, burrito, mexican food, chips, margaritas, guacamole, tortillas
   Ejemplo representativo:
   'We used to enjoy Tortilla Press. We recently went there for lunch. We found it very underwhelming. The guacamole is still good but my wife makes it better. Nothing had spice or zest. It seems that the...'

TÓPICO 2 (2,024 documentos)
   Palabras clave: food great, great food, great service, server, good food, attentive, love place, wonderful, food delicious, excellent food
  

## 7. Visualizaciones Interactivas

### Visualizaciones Disponibles

BERTopic proporciona múltiples visualizaciones interactivas:
1. **Mapa de tópicos**: Distribución espacial de tópicos en 2D
2. **Jerarquía de tópicos**: Relaciones jerárquicas entre tópicos
3. **Distribución de tópicos**: Tamaños relativos de cada tópico
4. **Palabras clave por tópico**: Importancia de palabras en cada tópico
5. **Similitud entre tópicos**: Matriz de distancias semánticas

### Utilidad de las Visualizaciones

- **Exploración interactiva**: Navegación intuitiva entre tópicos
- **Validación de coherencia**: Verificación visual de agrupaciones
- **Identificación de patrones**: Detección de relaciones temáticas
- **Presentación de resultados**: Comunicación efectiva de hallazgos

### Guardado de Visualizaciones

Las visualizaciones se guardarán en formato PNG.

In [10]:
print("Generando visualizaciones de BERTopic...")

import os
os.makedirs("../../figures/complete_analysis", exist_ok=True)

# 1. Visualización de tópicos en 2D
print("Creando mapa de tópicos...")
try:
    topics_viz = topic_model.visualize_topics()
    topics_viz.show()
    topics_viz.write_image("../../figures/complete_analysis/topic_model_topics_map.png")
    print("Mapa de tópicos guardado en: ../../figures/complete_analysis/topic_model_topics_map.png")
except Exception as e:
    print(f"Error al crear mapa de tópicos: {e}")

# 2. Distribución de documentos por tópico
print("\nCreando gráfico de distribución de tópicos...")
try:
    barchart = topic_model.visualize_barchart(top_k_topics=15)
    barchart.show()
    barchart.write_image("../../figures/complete_analysis/topic_model_barchart.png")
    print("Gráfico de barras guardado en: ../../figures/complete_analysis/topic_model_barchart.png")
except Exception as e:
    print(f"Error al crear gráfico de barras: {e}")

# 3. Visualización de jerarquía de tópicos
print("\nCreando jerarquía de tópicos...")
try:
    hierarchical_topics = topic_model.hierarchical_topics(documents)
    hierarchy_viz = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
    hierarchy_viz.show()
    hierarchy_viz.write_image("../../figures/complete_analysis/topic_model_hierarchy.png")
    print("Jerarquía de tópicos guardada en: ../../figures/complete_analysis/topic_model_hierarchy.png")
except Exception as e:
    print(f"Error al crear jerarquía: {e}")

# 4. Mapa de calor de similitud entre tópicos
print("\nCreando mapa de calor de similitud...")
try:
    similarity_viz = topic_model.visualize_heatmap()
    similarity_viz.show()
    similarity_viz.write_image("../../figures/complete_analysis/topic_model_heatmap.png")
    print("Mapa de calor guardado en: ../../figures/complete_analysis/topic_model_heatmap.png")
except Exception as e:
    print(f"Error al crear mapa de calor: {e}")

# 5. Análisis de palabras clave para un tópico específico
print("\nCreando análisis de palabras clave para tópico principal...")
try:
    main_topic = topic_info[topic_info['Topic'] != -1].iloc[0]['Topic']
    terms_viz = topic_model.visualize_term_rank(topics=[main_topic])
    terms_viz.show()
    terms_viz.write_image("../../figures/complete_analysis/topic_model_terms.png")
    print(f"Análisis de términos del tópico {main_topic} guardado en: ../../figures/complete_analysis/topic_model_terms.png")
except Exception as e:
    print(f"Error al crear análisis de términos: {e}")

print("\nTodas las visualizaciones han sido generadas y guardadas en formato PNG.")
print("Las imágenes están disponibles en: ../../figures/complete_analysis/")

Generando visualizaciones de BERTopic...
Creando mapa de tópicos...


Mapa de tópicos guardado en: ../../figures/complete_analysis/topic_model_topics_map.png

Creando gráfico de distribución de tópicos...
Error al crear gráfico de barras: BERTopic.visualize_barchart() got an unexpected keyword argument 'top_k_topics'

Creando jerarquía de tópicos...


100%|██████████| 69/69 [00:00<00:00, 512.67it/s]


Jerarquía de tópicos guardada en: ../../figures/complete_analysis/topic_model_hierarchy.png

Creando mapa de calor de similitud...


Mapa de calor guardado en: ../../figures/complete_analysis/topic_model_heatmap.png

Creando análisis de palabras clave para tópico principal...


Análisis de términos del tópico 0 guardado en: ../../figures/complete_analysis/topic_model_terms.png

Todas las visualizaciones han sido generadas y guardadas en formato PNG.
Las imágenes están disponibles en: ../../figures/complete_analysis/


## 8. Análisis de Tópicos por Sentimiento

### Correlación entre Tópicos y Sentimientos

Esta sección combina los resultados del modelado de tópicos con el análisis de sentimientos para descubrir:
- **Tópicos positivos**: Aspectos que generan satisfacción en restaurantes
- **Tópicos negativos**: Problemas frecuentes mencionados en reviews críticas
- **Tópicos neutrales**: Aspectos descriptivos sin carga emocional clara
- **Distribución emocional**: Cómo se distribuyen los sentimientos en cada tópico

### Metodología de Análisis

Analizaremos:
1. **Distribución de sentimientos por tópico**: Proporción de POSITIVE/NEGATIVE/NEUTRAL
2. **Tópicos más polarizados**: Aquellos con mayor sesgo hacia un sentimiento
3. **Tópicos equilibrados**: Aquellos con distribución uniforme de sentimientos
4. **Palabras clave contextualizadas**: Interpretación semántica por sentimiento

### Aplicaciones para el Negocio

Este análisis permite:
- Identificar fortalezas y debilidades temáticas
- Priorizar áreas de mejora basadas en feedback negativo
- Potenciar aspectos que generan satisfacción
- Desarrollar estrategias diferenciadas por tipo de experiencia

In [11]:
# Análisis de la relación entre tópicos y sentimientos
print("ANÁLISIS DE TÓPICOS POR SENTIMIENTO")
print("=" * 45)

# Crear DataFrame combinando tópicos y sentimientos
df_analysis = df_sample.copy()
df_analysis['topic'] = topics
df_analysis['topic_probability'] = [max(prob) if prob is not None else 0 for prob in probabilities]

# Filtrar solo documentos asignados a tópicos (excluir outliers)
df_topics = df_analysis[df_analysis['topic'] != -1].copy()

print(f"Documentos incluidos en el análisis: {len(df_topics):,}")
print(f"Documentos excluidos (outliers): {(df_analysis['topic'] == -1).sum():,}")

# Calcular distribución de sentimientos por tópico
sentiment_by_topic = df_topics.groupby(['topic', 'predicted_sentiment']).size().unstack(fill_value=0)
sentiment_by_topic_pct = sentiment_by_topic.div(sentiment_by_topic.sum(axis=1), axis=0) * 100

# Agregar información de conteos totales
sentiment_by_topic['Total'] = sentiment_by_topic.sum(axis=1)
sentiment_by_topic = sentiment_by_topic.sort_values('Total', ascending=False)

print(f"\nDISTRIBUCIÓN DE SENTIMIENTOS POR TÓPICO (Top 10)")
print("-" * 55)

# Mostrar los top 10 tópicos con su distribución de sentimientos
top_10_topics = sentiment_by_topic.head(10)
for topic_id in top_10_topics.index:
    total_docs = top_10_topics.loc[topic_id, 'Total']
    
    # Obtener palabras clave del tópico
    topic_words = topic_model.get_topic(topic_id)
    if topic_words:
        keywords = [word for word, _ in topic_words[:5]]
        keywords_str = ', '.join(keywords)
    else:
        keywords_str = "N/A"
    
    print(f"\nTÓPICO {topic_id} ({total_docs:,} docs)")
    print(f"   Palabras clave: {keywords_str}")
    
    # Mostrar distribución de sentimientos
    if 'POSITIVE' in sentiment_by_topic.columns:
        pos_pct = sentiment_by_topic_pct.loc[topic_id, 'POSITIVE']
        neg_pct = sentiment_by_topic_pct.loc[topic_id, 'NEGATIVE'] if 'NEGATIVE' in sentiment_by_topic_pct.columns else 0
        neu_pct = sentiment_by_topic_pct.loc[topic_id, 'NEUTRAL'] if 'NEUTRAL' in sentiment_by_topic_pct.columns else 0
        
        print(f"   Positivo: {pos_pct:.1f}% | Negativo: {neg_pct:.1f}% | Neutral: {neu_pct:.1f}%")
        
        # Clasificar el tópico basado en sentimiento dominante
        if pos_pct > 60:
            topic_type = "POSITIVO"
        elif neg_pct > 40:
            topic_type = "NEGATIVO"
        else:
            topic_type = "MIXTO"
        print(f"   Clasificación: {topic_type}")

# Identificar tópicos más positivos y negativos
print(f"\nTÓPICOS MÁS POSITIVOS")
print("-" * 25)
if 'POSITIVE' in sentiment_by_topic_pct.columns:
    most_positive = sentiment_by_topic_pct.sort_values('POSITIVE', ascending=False).head(5)
    for topic_id in most_positive.index:
        pos_pct = most_positive.loc[topic_id, 'POSITIVE']
        total = sentiment_by_topic.loc[topic_id, 'Total']
        topic_words = topic_model.get_topic(topic_id)
        keywords = [word for word, _ in topic_words[:3]] if topic_words else ["N/A"]
        print(f"Tópico {topic_id}: {pos_pct:.1f}% positivo ({total:,} docs) - {', '.join(keywords)}")

print(f"\nTÓPICOS MÁS NEGATIVOS")
print("-" * 25)
if 'NEGATIVE' in sentiment_by_topic_pct.columns:
    most_negative = sentiment_by_topic_pct.sort_values('NEGATIVE', ascending=False).head(5)
    for topic_id in most_negative.index:
        neg_pct = most_negative.loc[topic_id, 'NEGATIVE']
        total = sentiment_by_topic.loc[topic_id, 'Total']
        topic_words = topic_model.get_topic(topic_id)
        keywords = [word for word, _ in topic_words[:3]] if topic_words else ["N/A"]
        print(f"Tópico {topic_id}: {neg_pct:.1f}% negativo ({total:,} docs) - {', '.join(keywords)}")

print(f"\nAnálisis de tópicos por sentimiento completado!")

ANÁLISIS DE TÓPICOS POR SENTIMIENTO
Documentos incluidos en el análisis: 29,418
Documentos excluidos (outliers): 14,575

DISTRIBUCIÓN DE SENTIMIENTOS POR TÓPICO (Top 10)
-------------------------------------------------------

TÓPICO 0 (5,888 docs)
   Palabras clave: minutes, asked, manager, told, rude
   Positivo: 8.3% | Negativo: 87.3% | Neutral: 4.4%
   Clasificación: NEGATIVO

TÓPICO 1 (2,395 docs)
   Palabras clave: tacos, mexican, taco, salsa, burrito
   Positivo: 71.8% | Negativo: 27.2% | Neutral: 1.0%
   Clasificación: POSITIVO

TÓPICO 2 (2,024 docs)
   Palabras clave: food great, great food, great service, server, good food
   Positivo: 95.2% | Negativo: 4.8% | Neutral: 0.0%
   Clasificación: POSITIVO

TÓPICO 3 (1,884 docs)
   Palabras clave: pizza, crust, pizzas, best pizza, pepperoni
   Positivo: 75.7% | Negativo: 23.4% | Neutral: 0.9%
   Clasificación: POSITIVO

TÓPICO 4 (1,201 docs)
   Palabras clave: sushi, roll, rolls, best sushi, hibachi
   Positivo: 69.9% | Negativo: 2

## 9. Guardado de Resultados

### Exportación de Datos

Guardaremos los resultados del modelado de tópicos en múltiples formatos:
- **Modelo entrenado**: Archivo pickle para reutilización
- **Asignaciones de tópicos**: CSV con tópicos por documento
- **Información de tópicos**: CSV con palabras clave y estadísticas
- **Análisis combinado**: CSV con tópicos, sentimientos y metadata

### Utilidad de los Archivos Guardados

Los resultados exportados permiten:
- **Reutilización del modelo**: Aplicar a nuevos documentos sin reentrenar
- **Análisis posterior**: Integración con otras herramientas y análisis
- **Reproducibilidad**: Garantizar consistencia en análisis futuros
- **Presentación**: Datos listos para dashboards y reportes

### Estructura de Archivos

Los archivos se organizan en:
- `models/topic_modeling/`: Modelos entrenados
- `data/results/`: Datasets con resultados de análisis
- `figures/complete_analysis/`: Visualizaciones interactivas

In [12]:
import os
import numpy as np
import pandas as pd

print("GUARDANDO RESULTADOS DEL MODELADO DE TÓPICOS")
print("=" * 50)

# Crear directorios si no existen
os.makedirs("../../models/topic_modeling", exist_ok=True)
os.makedirs("../../data/results", exist_ok=True)

# 1. Guardar el modelo entrenado
print("Guardando modelo BERTopic entrenado...")
try:
    topic_model.save("../../models/topic_modeling/bertopic_model")
    print("Modelo guardado en: ../../models/topic_modeling/bertopic_model/")
except Exception as e:
    print(f"Error al guardar modelo: {e}")

# 2. Guardar información de tópicos
print("\nGuardando información de tópicos...")
try:
    topic_info.to_csv("../../data/results/topic_model_info.csv", index=False)
    print("Información de tópicos guardada en: ../../data/results/topic_model_info.csv")
except Exception as e:
    print(f"Error al guardar información de tópicos: {e}")

# 3. Guardar asignaciones de tópicos por documento
print("\nGuardando asignaciones de tópicos...")
try:
    # Asegurar que topics es un array de numpy para operaciones vectorizadas
    topics = np.array(topics)
    
    topic_assignments = pd.DataFrame({
        'review_id': df_sample['review_id'],
        'business_id': df_sample['business_id'],
        'user_id': df_sample['user_id'],
        'stars': df_sample['stars'],
        'predicted_sentiment': df_sample['predicted_sentiment'],
        'confidence': df_sample['confidence'],
        'topic': topics,
        'topic_probability': [max(prob) if prob is not None else 0 for prob in probabilities],
        'text_length': df_sample['text'].str.len(),
        'date': df_sample['date']
    })
    
    topic_assignments.to_csv("../../data/results/topic_assignments.csv", index=False)
    print(f"Asignaciones guardadas en: ../../data/results/topic_assignments.csv")
    print(f"   Registros guardados: {len(topic_assignments):,}")
except Exception as e:
    print(f"Error al guardar asignaciones: {e}")

# 4. Guardar distribución de sentimientos por tópico
print("\nGuardando análisis de sentimientos por tópico...")
try:
    # Combinar información de distribución de sentimientos
    sentiment_analysis_df = sentiment_by_topic.copy()
    sentiment_analysis_df['Topic'] = sentiment_analysis_df.index

    # Agregar palabras clave
    sentiment_analysis_df['Keywords'] = sentiment_analysis_df['Topic'].apply(
        lambda topic_id: ', '.join([word for word, _ in topic_model.get_topic(topic_id)[:10]]) 
        if topic_model.get_topic(topic_id) else 'N/A'
    )

    # Agregar porcentajes si existen
    for col in ['POSITIVE', 'NEGATIVE', 'NEUTRAL']:
        if col in sentiment_by_topic_pct.columns:
            sentiment_analysis_df[f'{col}_pct'] = sentiment_by_topic_pct[col]

    sentiment_analysis_df.to_csv("../../data/results/topic_sentiment_analysis.csv", index=False)
    print("Análisis de sentimientos guardado en: ../../data/results/topic_sentiment_analysis.csv")
except Exception as e:
    print(f"Error al guardar análisis de sentimientos: {e}")

# 5. Guardar estadísticas resumen
print("\nGuardando estadísticas resumen...")
try:
    # Asegurar que topics sigue como array de numpy para contar correctamente
    num_assigned = (topics != -1).sum()
    num_outliers = (topics == -1).sum()
    total_docs = len(documents)
    num_topics = num_topics  # Debe estar definido antes
    summary_stats = {
        'total_documents': total_docs,
        'documents_assigned': num_assigned,
        'outliers': num_outliers,
        'num_topics': num_topics,
        'avg_docs_per_topic': num_assigned / num_topics if num_topics > 0 else 0,
        'outlier_percentage': num_outliers / len(topics) * 100,
        'model_type': 'BERTopic',
        'embedding_model': 'all-MiniLM-L6-v2',
        'min_cluster_size': 50,
        'processing_time_minutes': elapsed_time / 60
    }

    summary_df = pd.DataFrame([summary_stats])
    summary_df.to_csv("../../data/results/topic_modeling_summary.csv", index=False)
    print("Estadísticas resumen guardadas en: ../../data/results/topic_modeling_summary.csv")
except Exception as e:
    print(f"Error al guardar estadísticas: {e}")

print(f"\nMODELADO DE TÓPICOS COMPLETADO EXITOSAMENTE!")
print("=" * 50)
print(f"Resultados principales:")
print(f"   - Tópicos descubiertos: {num_topics}")
print(f"   - Documentos procesados: {total_docs:,}")
print(f"   - Tiempo de procesamiento: {elapsed_time/60:.2f} minutos")
print(f"   - Documentos asignados: {num_assigned:,} ({num_assigned/len(topics)*100:.1f}%)")
print(f"   - Outliers: {num_outliers:,} ({num_outliers/len(topics)*100:.1f}%)")

print(f"\nArchivos generados:")
print(f"   - Modelo: ../../models/topic_modeling/bertopic_model/")
print(f"   - Datos: ../../data/results/topic_*.csv")
print(f"   - Visualizaciones: ../../figures/complete_analysis/topic_model_*.html / *.png")

print(f"\nPróximos pasos sugeridos:")
print(f"   1. Explorar visualizaciones interactivas en navegador")
print(f"   2. Analizar tópicos más positivos/negativos")
print(f"   3. Correlacionar tópicos con ratings de restaurantes")
print(f"   4. Integrar resultados en dashboard de Streamlit")
print(f"   5. Documentar hallazgos principales para el TFM")

2025-07-11 01:16:51,538 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


GUARDANDO RESULTADOS DEL MODELADO DE TÓPICOS
Guardando modelo BERTopic entrenado...
Modelo guardado en: ../../models/topic_modeling/bertopic_model/

Guardando información de tópicos...
Información de tópicos guardada en: ../../data/results/topic_model_info.csv

Guardando asignaciones de tópicos...
Asignaciones guardadas en: ../../data/results/topic_assignments.csv
   Registros guardados: 43,993

Guardando análisis de sentimientos por tópico...
Análisis de sentimientos guardado en: ../../data/results/topic_sentiment_analysis.csv

Guardando estadísticas resumen...
Estadísticas resumen guardadas en: ../../data/results/topic_modeling_summary.csv

MODELADO DE TÓPICOS COMPLETADO EXITOSAMENTE!
Resultados principales:
   - Tópicos descubiertos: 70
   - Documentos procesados: 43,993
   - Tiempo de procesamiento: 0.98 minutos
   - Documentos asignados: 29,418 (66.9%)
   - Outliers: 14,575 (33.1%)

Archivos generados:
   - Modelo: ../../models/topic_modeling/bertopic_model/
   - Datos: ../../data